### En este código se realiza la instalación de la conexión a la API de mercado libre y el análisis expliatorio de datos
 1. Acceder a la API

Para garantizar una adecuada organización y gestión de las dependencias que usaremos, configuaremos un entorno virtual en el mismo repositorio (ver archivo README.md)

In [16]:
!pip3 install requests pandas 

In [14]:
import sys 
import os 
print(sys.executable) #Verificamos que estamos corriendo el jupyter en el entorno virtual definido para ello.
os.getcwd() # veriicamos el directorio donde tenemos el repo local
sys.path.append('../similarity') #Agrego el paquete de python que cree para las funciones personalizadas

/Users/anfelipecb/GitHub/MeLi/MeLi_Challenge_Similarity/melienv/bin/python


In [ ]:
#Funciones paa obtener las categorias 

In [17]:
# Categrías de los ítems en marketplace
import requests
from similarity.config import API_URL
cats=requests.get(f'{API_URL}/sites/MLA/categories') #MLA hace referencia a argentina, que es el país escogido
cats.json()

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1500', 'name': 'Construcción'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas'},
 {'id': 'MLA1574', 'name': 'H

In [48]:
#items
import requests
cat_id='MLA1000' #  me voy con ML1000 que es en general electrónica, audio y video. 
#cat_id='MLA409810' #MLA409810 solo los auriculares bluetooth que vi, pero hay otros con otro_id
offset=0

url= f'{API_URL}/sites/MLA/search?category={cat_id}&offset={offset}'

request=requests.get(url)
items=request.json()



In [47]:
items
# Con estos primeros 50 datos vamos a armar una base de datos. 

{'site_id': 'MLA',
 'country_default_time_zone': 'GMT-03:00',
 'paging': {'total': 146762,
  'primary_results': 1000,
  'offset': 0,
  'limit': 50},
 'results': [{'id': 'MLA1114183052',
   'title': 'Auriculares Bluetooth In-ear Gamer F9-5 Negro Inalámbricos',
   'condition': 'new',
   'thumbnail_id': '938092-MLA45480677826_042021',
   'catalog_product_id': 'MLA16224063',
   'listing_type_id': 'gold_special',
   'permalink': 'https://www.mercadolibre.com.ar/auriculares-bluetooth-in-ear-gamer-f9-5-negro-inalambricos/p/MLA16224063',
   'buying_mode': 'buy_it_now',
   'site_id': 'MLA',
   'category_id': 'MLA3697',
   'domain_id': 'MLA-HEADPHONES',
   'thumbnail': 'http://http2.mlstatic.com/D_938092-MLA45480677826_042021-I.jpg',
   'currency_id': 'ARS',
   'order_backend': 1,
   'price': 5018,
   'original_price': None,
   'sale_price': None,
   'available_quantity': 500,
   'official_store_id': None,
   'use_thumbnail_id': True,
   'accepts_mercadopago': True,
   'shipping': {'store_pick_u

In [35]:
item=items['results'][0] # primer resultado
attributes=item['attributes'] #Dentro del diccionario de atributos encontramos caracteristicas útiles
attributes

[{'id': 'BRAND',
  'name': 'Marca',
  'value_id': '15375',
  'value_name': 'Energizer',
  'attribute_group_id': 'OTHERS',
  'attribute_group_name': 'Otros',
  'value_struct': None,
  'values': [{'id': '15375',
    'name': 'Energizer',
    'struct': None,
    'source': 1}],
  'source': 1,
  'value_type': 'string'},
 {'id': 'ITEM_CONDITION',
  'name': 'Condición del ítem',
  'value_id': '2230284',
  'value_name': 'Nuevo',
  'attribute_group_id': 'OTHERS',
  'attribute_group_name': 'Otros',
  'value_struct': None,
  'values': [{'id': '2230284',
    'name': 'Nuevo',
    'struct': None,
    'source': 6808261514773724}],
  'source': 6808261514773724,
  'value_type': 'list'},
 {'id': 'LINE',
  'name': 'Línea',
  'value_id': '18167',
  'value_name': 'MAX',
  'attribute_group_id': 'OTHERS',
  'attribute_group_name': 'Otros',
  'value_struct': None,
  'values': [{'id': '18167', 'name': 'MAX', 'struct': None, 'source': 1}],
  'source': 1,
  'value_type': 'string'},
 {'id': 'MODEL',
  'name': 'Mod

In [38]:
#Intentemos cargar todo a la bd
import pandas as pd
from pandas import json_normalize
import requests
cat_id='MLA1000'
offset=0
url= f'{API_URL}/sites/MLA/search?category={cat_id}&offset={offset}'

request=requests.get(url)
data=request.json()
products_list= data ['results']
df=json_normalize(products_list)
# vamos a extraer atributos específicos 
for attr_name in ['BRAND', 'LINE', 'MODEL', 'PACKAGE_LENGTH', 'PACKAGE_WEIGHT']:
    # Creamos una nueva columna para cada atributo
    df[attr_name.lower()] = df['attributes'].apply(lambda x: next((item['value_name'] for item in x if item['id'] == attr_name), None))
# Ahora podemos eliminar la columna original 'attributes' 
df.drop(columns=['attributes'], inplace=True)

print(df.head())

              id  ... package_weight
0  MLA1470370644  ...          240 g
1  MLA1568419348  ...        14540 g
2  MLA1114183052  ...          100 g
3  MLA1407677669  ...          920 g
4  MLA1411155053  ...         4600 g

[5 rows x 47 columns]


In [41]:
df.head()

,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,domain_id,thumbnail,currency_id,order_backend,price,original_price,sale_price,available_quantity,official_store_id,use_thumbnail_id,accepts_mercadopago,stop_time,winner_item_id,catalog_listing,discounts,promotions,inventory_id,shipping.store_pick_up,shipping.free_shipping,shipping.logistic_type,shipping.mode,shipping.tags,shipping.benefits,shipping.promise,seller.id,seller.nickname,installments.quantity,installments.amount,installments.rate,installments.currency_id,official_store_name,differential_pricing.id,brand,line,model,package_length,package_weight
0,MLA1470370644,20 X Pilas Aaa Energizer Max E92 Pilas Alcalin...,new,976056-MLA74192071235_012024,MLA22288649,gold_special,https://www.mercadolibre.com.ar/20-x-pilas-aaa...,buy_it_now,MLA,MLA6384,MLA-CELL_BATTERIES,http://http2.mlstatic.com/D_976056-MLA74192071...,ARS,1,12881.62,NaN,None,250,NaN,True,True,2043-08-06T04:00:00.000Z,None,True,None,[],GEUO71385,False,False,fulfillment,me2,"[fulfillment, MLA-chg-threshold-ago-22, self_s...",None,None,702150429,GLOBALNAVET,6,3411.91,58.92,ARS,NaN,NaN,Energizer,MAX,E92,12.2 cm,240 g
1,MLA1568419348,Smart Google Tv Tcl Qled 55c645 4k Uhd,new,667349-MLU71651838914_092023,MLA26994250,gold_pro,https://www.mercadolibre.com.ar/smart-google-t...,buy_it_now,MLA,MLA1002,MLA-TELEVISIONS,http://http2.mlstatic.com/D_667349-MLU71651838...,ARS,2,699999.00,889999.0,None,1,1938.0,True,True,2043-11-12T04:00:00.000Z,None,True,None,[],JKDT81176,False,True,fulfillment,me2,"[fulfillment, mandatory_free_shipping]",None,None,608846165,MERCADOLIBRE ELECTRONICA_AR,3,233333.00,0.00,ARS,Mercado Libre Electronica,35713320.0,TCL,QLED,L55C645,132 cm,14540 g
2,MLA1114183052,Auriculares Bluetooth In-ear Gamer F9-5 Negro ...,new,938092-MLA45480677826_042021,MLA16224063,gold_special,https://www.mercadolibre.com.ar/auriculares-bl...,buy_it_now,MLA,MLA3697,MLA-HEADPHONES,http://http2.mlstatic.com/D_938092-MLA45480677...,ARS,3,5018.00,NaN,None,500,NaN,True,True,2041-11-24T04:00:00.000Z,None,True,None,[],YZGH48802,False,False,fulfillment,me2,"[fulfillment, self_service_out]",None,None,82564913,GANGAFAN,6,1329.10,58.92,ARS,NaN,NaN,Shenzhen Yihaotong,Bluetooth,F9-5,11 cm,100 g
3,MLA1407677669,Parlante Jbl Flip 6 Portátil Con Bluetooth Roj...,new,690313-MLA74116300578_012024,MLA18930465,gold_pro,https://www.mercadolibre.com.ar/parlante-jbl-f...,buy_it_now,MLA,MLA8618,MLA-SPEAKERS,http://http2.mlstatic.com/D_690313-MLA74116300...,ARS,4,229999.00,NaN,None,1,1938.0,True,True,2044-02-03T04:00:00.000Z,None,True,None,[],JFQN19213,False,True,fulfillment,me2,"[fulfillment, mandatory_free_shipping]",None,None,608846165,MERCADOLIBRE ELECTRONICA_AR,3,76666.33,0.00,ARS,Mercado Libre Electronica,35713320.0,JBL,Flip,Flip 6,21.6 cm,920 g
4,MLA1411155053,Smart Tv Led Android Tv 32 Rca R32andf Hd Blue...,new,973344-MLU72648243726_112023,MLA27977446,gold_pro,https://www.mercadolibre.com.ar/smart-tv-led-a...,buy_it_now,MLA,MLA1002,MLA-TELEVISIONS,http://http2.mlstatic.com/D_973344-MLU72648243...,ARS,5,229999.00,279999.0,None,200,NaN,True,True,2044-02-28T14:35:44.000Z,None,True,None,[],TBUK22950,False,True,fulfillment,me2,"[fulfillment, mandatory_free_shipping]",None,None,608846165,MERCADOLIBRE ELECTRONICA_AR,3,76666.33,0.00,ARS,NaN,35713320.0,RCA,R32AND,R32AND,81 cm,4600 g


In [49]:
#La funcion anterior funcionó perfectamente, pero ahora vamos a generalizarla para hacer solicitudes sucesivas
def scrap_products(category_id, api_url, limit=10000):
    """
    Obtiene los productos de la categoría indicada a partir de la API de MercadoLibre API para una categoría específica y hasta un limite determinado 
    (default= 10000) 
    
    Parametros:
    - category_id: El ID de la categoría de la que se sacan los productos.
    - api_url: La URL base de la API MELI.
    - limit: Numero maximo de productos para reunir.
    
    Devuelve (return):
    - Data frame de pandas conteniendo todos los productos hasta el limite especificado
    """
    offset = 0
    products_list = []  #Lista vaciía de productos 
    
    while offset < limit:
        url = f'{api_url}/sites/MLA/search?category={category_id}&offset={offset}'
        response = requests.get(url)
        data = response.json()
        
        # Revisa que no hay mas productos para traer
        if not data['results']:
            break
        
        products_list.extend(data['results'])
        
        # actualización del offset
        offset += len(data['results']) #iteracion de 50 en 50 en la práctica, pero más flexible, por si la última página no tiene los 50 sino menos
    #Normalizamos 
    df = json_normalize(products_list)
    
    # Los atributos que me parecen importantes 
        # Aquí: df.apply aplicara a cada fila la funcion lambda para transformar la estructura de datos anidada a la base que estamos armando
    for attr_name in ['BRAND', 'LINE', 'MODEL', 'PACKAGE_LENGTH', 'PACKAGE_WEIGHT']:
        df[attr_name.lower()] = df.apply(
            lambda row: next(
                (attr['value_name'] for attr in row['attributes'] if attr['id'] == attr_name), 
                None
            ), 
            axis=1
        )
    # Quitamos atributo que no voy a usar para hacer menos espacio
    df.drop(columns=['attributes'], inplace=True)
    return df


In [51]:
# Uso de la función
# API_URL viene del SETUP que configuté
CATEGORY_ID = 'MLA1000'  # Cambiar por la categoría deseada
LIMIT = 10000  # Cambiar según sea necesario

df_products = scrap_products(CATEGORY_ID, API_URL, LIMIT)
print(df_products.head())

KeyError: 'results'

In [30]:
# Categorías de segundo nivel
url= f'{API_URL}/categories/{cat_id}'
request_f=requests.get(url)

In [33]:
request_f.json() # el 409810 es el de items de audio (el producto que escogí)

{'id': 'MLA1000',
 'name': 'Electrónica, Audio y Video',
 'picture': 'https://http2.mlstatic.com/storage/categories-api/images/943ec641-717e-49cb-8a34-2f40ba367f5a.png',
 'permalink': 'https://www.mercadolibre.com.ar/c/electronica-audio-y-video',
 'total_items_in_this_category': 593863,
 'path_from_root': [{'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'}],
 'children_categories': [{'id': 'MLA3690',
   'name': 'Accesorios para Audio y Video',
   'total_items_in_this_category': 16006},
  {'id': 'MLA431414',
   'name': 'Accesorios para TV',
   'total_items_in_this_category': 15716},
  {'id': 'MLA409810', 'name': 'Audio', 'total_items_in_this_category': 206161},
  {'id': 'MLA5054', 'name': 'Cables', 'total_items_in_this_category': 27748},
  {'id': 'MLA11830',
   'name': 'Componentes Electrónicos',
   'total_items_in_this_category': 174981},
  {'id': 'MLA4632',
   'name': 'Controles Remotos',
   'total_items_in_this_category': 14373},
  {'id': 'MLA352294',
   'name': 'Drones y Acceso